### Preparacion del dataset

Del estudio realizado en el [Data-analisis](data-analisis.ipynb)  se obtiene que los parametros importantes para modelar son: 'Species', 'Light_ISF', 'Soil', 'Sterile', 'Conspecific', 'AMF', 'EMF', 'Phenolics', 'Lignin', 'NSC' y 'Event', siendo este ultimo el target.

In [1]:
import pandas as pd

dataset = pd.read_csv("../dataset/Tree_Data.csv")

In [2]:
dataset_modif = dataset[['Species', 'Light_ISF', 'Soil', 'Sterile', 
                         'Conspecific', 'AMF', 'EMF', 'Phenolics', 'Lignin', 'NSC', 'Event']]

dataset_modif.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2783 entries, 0 to 2782
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Species      2783 non-null   object 
 1   Light_ISF    2783 non-null   float64
 2   Soil         2783 non-null   object 
 3   Sterile      2783 non-null   object 
 4   Conspecific  2783 non-null   object 
 5   AMF          2783 non-null   float64
 6   EMF          1283 non-null   float64
 7   Phenolics    2783 non-null   float64
 8   Lignin       2783 non-null   float64
 9   NSC          2783 non-null   float64
 10  Event        2782 non-null   float64
dtypes: float64(7), object(4)
memory usage: 239.3+ KB


Un solo Event es null, lo elimino y los nan del EMF los paso a 0

In [3]:
dataset_modif = dataset_modif.dropna(subset=["Event"])
dataset_modif.fillna(value={"EMF": 0}, inplace=True)

dataset_modif

,Species,Light_ISF,Soil,Sterile,Conspecific,AMF,EMF,Phenolics,Lignin,NSC,Event
0,Acer saccharum,0.106,Prunus serotina,Non-Sterile,Heterospecific,22.00,0.00,-0.56,13.86,12.15,1.0
1,Quercus alba,0.106,Quercus rubra,Non-Sterile,Heterospecific,15.82,31.07,5.19,20.52,19.29,0.0
2,Quercus rubra,0.106,Prunus serotina,Non-Sterile,Heterospecific,24.45,28.19,3.36,24.74,15.01,1.0
3,Acer saccharum,0.080,Prunus serotina,Non-Sterile,Heterospecific,22.23,0.00,-0.71,14.29,12.36,1.0
4,Acer saccharum,0.060,Prunus serotina,Non-Sterile,Heterospecific,21.15,0.00,-0.58,10.85,11.20,1.0
...,...,...,...,...,...,...,...,...,...,...,...
2777,Quercus alba,0.122,Quercus rubra,Non-Sterile,Heterospecific,10.89,39.00,5.53,21.44,18.99,1.0
2778,Prunus serotina,0.111,Populus grandidentata,Non-Sterile,Heterospecific,40.89,0.00,0.83,9.15,11.88,1.0
2779,Quercus alba,0.118,Acer rubrum,Non-Sterile,Heterospecific,15.47,32.82,4.88,19.01,23.50,1.0
2780,Quercus alba,0.118,Quercus rubra,Non-Sterile,Heterospecific,11.96,37.67,5.51,21.13,19.10,1.0


In [5]:
# La guardamos en un csv

dataset_modif.to_csv("../dataset/data_Modif.csv", index=False)

In [8]:
# probamos que se haya guardado correctamente
prueba_dataset_modif = pd.read_csv("../dataset/data_Modif.csv")

prueba_dataset_modif.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2782 entries, 0 to 2781
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Species      2782 non-null   object 
 1   Light_ISF    2782 non-null   float64
 2   Soil         2782 non-null   object 
 3   Sterile      2782 non-null   object 
 4   Conspecific  2782 non-null   object 
 5   AMF          2782 non-null   float64
 6   EMF          2782 non-null   float64
 7   Phenolics    2782 non-null   float64
 8   Lignin       2782 non-null   float64
 9   NSC          2782 non-null   float64
 10  Event        2782 non-null   float64
dtypes: float64(7), object(4)
memory usage: 239.2+ KB


### Modelando

In [1]:
import pandas as pd

dataset = pd.read_csv("../dataset/data_Modif.csv")

# dropeo cualquier observacion con algun valor nulo
dataset =  dataset.dropna()

X = dataset.drop(['Event'], axis = 1)
y = dataset['Event']

print("X: " + str(X.shape))
print("y: " + str(y.shape))

X: (2782, 10)
y: (2782,)


In [26]:
# utilizo ColumnTransformer para manejar las columnas numericas y categoricas
# luego un pipeline para acoplar model selectot 
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

# defino las columnas numericas y categoricas
num_cols = ['Light_ISF', 'AMF', 'EMF', 'Phenolics', 'Lignin', 'NSC']
cat_cols = ['Species', 'Soil', 'Sterile', 'Conspecific']
 
# armo el columnTransformer
col_trans = ColumnTransformer([
    ('scalador_col_num', StandardScaler(), num_cols),
    ('one-hot_cat_num', OneHotEncoder(), cat_cols)
    ],
    remainder='drop') # descarto el resto de columnas

# armo el pipeline
estimador_LR = Pipeline([
    ('manejo de columnas', col_trans),
    ('core_model', LogisticRegression())
])

display(estimador_LR)

Pipeline(steps=[('manejo de columnas',
                 ColumnTransformer(transformers=[('scalador_col_num',
                                                  StandardScaler(),
                                                  ['Light_ISF', 'AMF', 'EMF',
                                                   'Phenolics', 'Lignin',
                                                   'NSC']),
                                                 ('one-hot_cat_num',
                                                  OneHotEncoder(),
                                                  ['Species', 'Soil', 'Sterile',
                                                   'Conspecific'])])),
                ('core_model', LogisticRegression())])

In [3]:
# hago validacion cruzada
from sklearn.model_selection import cross_validate
import numpy as np

results_LR = cross_validate(estimador_LR, X, y, cv=10, return_train_score=True)

train_score_LR = np.mean(results_LR['train_score'])
test_score_LR = np.mean(results_LR['test_score'])

print(f'Train Score: {round(train_score_LR, 6)}')
print(f'Test Score: {round(test_score_LR, 6)}')

Train Score: 0.80262
Test Score: 0.802655


Se obtiene un buen score, tanto en el train como en el test (indicaria que no hay overfitting)

### Comparacion con otros estimadores

Se ha realizado un modelo con LogisticRegression, a continuacion se realizara otros dos modelos, con RandomForestClassifier y con GradientBoostingClassifier, y se compararan para ver en cual se obtiene mejor Score

In [4]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

# Solo tengo que armar un nuevo pipeline, ya que el ColumnTransformer se mantiene igual 

# Pipeline para el RandomForestClassifier
estimador_RFC = Pipeline([
    ('manejo de columnas', col_trans),
    ('core_model', RandomForestClassifier(random_state= 42))
])

# Pipeline para el GradientBoostingClassifier
estimador_GBC = Pipeline([
    ('manejo de columnas', col_trans),
    ('core_model', GradientBoostingClassifier(random_state= 42))
])


In [5]:
estimadores = {
    'LogisticRegression' : estimador_LR,
    'RandomForestClassifier' : estimador_RFC,
    'GradientBoostingClassifier' : estimador_GBC
}

for estimador_nombre, estimador in estimadores.items():
    resultados = cross_validate(estimador, X, y, cv=10, return_train_score=True)

    train_score = np.mean(resultados['train_score'])
    test_score = np.mean(resultados['test_score'])
    fit_time = np.mean(resultados['fit_time'])

    print(str(estimador_nombre) + ':')
    print(f'        Train Score: {round(train_score, 6)}')
    print(f'        Test Score: {round(test_score, 6)}')
    print(f'        Fit time: {round(fit_time, 6)}')
    print('---------------------------------') 


LogisticRegression:
        Train Score: 0.80262
        Test Score: 0.802655
        Fit time: 0.043764
---------------------------------
RandomForestClassifier:
        Train Score: 0.95395
        Test Score: 0.799021
        Fit time: 0.865123
---------------------------------
GradientBoostingClassifier:
        Train Score: 0.864126
        Test Score: 0.818456
        Fit time: 0.964637
---------------------------------


El GradientBoostingClassifier consigue una mejora frente al LogisticRegression. El RandomForestClassifier tambien lo consigue pero, que en train haya conseguido una mejora y en el test hay empeorado, indicaria un posible sobre ajuste.

Si miramos el tiempo el LogisticRegression aun posee un buen score y su tiempo es un 95% menos que el GradientBoostingClassifier

### Mejora de hiperparametros

Vamos a buscar la mejor configuracion para cada modelo

In [60]:
# parametros a optimizar
# los parametros hay que escribirlos de la forma <nombre en el pipeline>__<parameter>

# LogisticRegression
# segun la documentacion, lo solver recomendados para el tamaño del dataset son lbfgs y liblinear
# lbfgs al ser el default ya fue evaluado, por lo que queda el liblinear con las penalizaciones

parameters_LR = {
    'core_model__solver':['liblinear'],
    'core_model__penalty':['l1', 'l2']
}

# RandomForestClassifier

parametros_RFC = {
    'core_model__n_estimators' : range(4,200), # numero de estimadores (arboles) del random forest
    'core_model__criterion' : ['gini', 'entropy', 'log_loss'], # medida de calidad (que tan bueno fue) el split de los arboles
    'core_model__max_depth' : range(2,11) # que tan profundo es el arbol
}

# GradientBoostingClassifier

parametros_GBC = {
    'core_model__n_estimators': range(100,200,10),
    'core_model__max_depth' : range(2,11)
} 

In [61]:
from sklearn.model_selection import RandomizedSearchCV

estimadores_parametros = {
    'LogisticRegression' : [estimador_LR, parameters_LR],
    'RandomForestClassifier' : [estimador_RFC, parametros_RFC],
    'GradientBoostingClassifier' : [estimador_GBC, parametros_GBC]
}

mejores_estimadores_parametros = {
    'LogisticRegression' : 0,
    'RandomForestClassifier' : 0,
    'GradientBoostingClassifier' : 0
}

for estimador_nombre, estimador_parametro in estimadores_parametros.items():
    rand_est = RandomizedSearchCV(
        estimador_parametro[0], 
        estimador_parametro[1],
        n_iter= 20, 
        cv=3, 
        scoring="r2")
    
    rand_est.fit(X, y)

    mejores_estimadores_parametros[estimador_nombre] = rand_est.best_estimator_
    print('Encontrado mejor estimador para ' + str(estimador_nombre) + ':')
   
    resultados = cross_validate(rand_est.best_estimator_, X, y, cv=10, return_train_score=True)

    train_score = np.mean(resultados['train_score'])
    test_score = np.mean(resultados['test_score'])
    fit_time = np.mean(resultados['fit_time'])

    print(f'        Train Score: {round(train_score, 6)}')
    print(f'        Test Score: {round(test_score, 6)}')
    print(f'        Fit time: {round(fit_time, 6)}')
    print('---------------------------------')   

d:\Disco d\Milesi\01 - Personal\platzi\06 - Despliegue de Modelos de Machine Learning\practica-despliegue-modelos-ML\ent-model-cre\Lib\site-packages\sklearn\model_selection\_search.py:307: UserWarning: The total space of parameters 2 is smaller than n_iter=20. Running 2 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Encontrado mejor estimador para LogisticRegression:
        Train Score: 0.80262
        Test Score: 0.802655
        Fit time: 0.027111
---------------------------------
Encontrado mejor estimador para RandomForestClassifier:
        Train Score: 0.857177
        Test Score: 0.807306
        Fit time: 0.937041
---------------------------------
Encontrado mejor estimador para GradientBoostingClassifier:
        Train Score: 0.841321
        Test Score: 0.822062
        Fit time: 0.725796
---------------------------------


El GradientBoostingClassifier consigue una pequeña mejora, asi que me quedo con este. Como observacion, el LogisticRegression consigue un score algo menor pero tarda un 98% menos al momento del entrenamiento

In [62]:
display(mejores_estimadores_parametros['GradientBoostingClassifier'])

Pipeline(steps=[('manejo de columnas',
                 ColumnTransformer(transformers=[('scalador_col_num',
                                                  StandardScaler(),
                                                  ['Light_ISF', 'AMF', 'EMF',
                                                   'Phenolics', 'Lignin',
                                                   'NSC']),
                                                 ('one-hot_cat_num',
                                                  OneHotEncoder(),
                                                  ['Species', 'Soil', 'Sterile',
                                                   'Conspecific'])])),
                ('core_model',
                 GradientBoostingClassifier(max_depth=2, n_estimators=110,
                                            random_state=42))])

### Guardado del modelo

In [63]:
from sklearn.model_selection import train_test_split

estimador_elegido = mejores_estimadores_parametros['GradientBoostingClassifier']

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.30)

estimador_elegido.fit(X_train, y_train)

train_score_fin = estimador_elegido.score(X_train, y_train)
test_score_fin = estimador_elegido.score(X_test, y_test)

print(f'Train Score: {round(train_score_fin, 6)}')
print(f'Test Score: {round(test_score_fin, 6)}')

Train Score: 0.847458
Test Score: 0.82515


In [65]:
from joblib import dump

dump(estimador_elegido, '../model/model.pkl')

['../model/model.pkl']